In [1]:
import pandas as pd 
import numpy as np 
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

from sklearn.preprocessing import MinMaxScaler
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import TimeSeriesSplit

# Models
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.neural_network import MLPRegressor

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import warnings
warnings.filterwarnings('ignore')

import math

import datetime
from datetime import datetime as dt
from dateutil.relativedelta import *
from datetime import timedelta

In [2]:
def add_days(df, forecast_length):
    end_point = len(df)
    df1 = pd.DataFrame(index=range(forecast_length), columns=range(2))
    df1.columns = ['Close', 'Date']
    df = df.append(df1)
    df = df.reset_index(drop=True)
    x = df.at[end_point - 1, 'Date']
    x = pd.to_datetime(x, format='%Y-%m-%d')
    for i in range(forecast_length):
        df.at[df.index[end_point + i], 'Date'] = x + timedelta(days=1+i)
        df.at[df.index[end_point + i], 'Close'] = 0
    df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d')
    df = df.drop(['Date'], axis=1)
    return df

In [9]:
def forecasting(model,df1, forecast_length,target='Close'):
    df3 = df1[[target, 'Date']]
    df3 = add_days(df3, forecast_length)
    finaldf = df1.drop('Date', axis=1)
    finaldf = finaldf.reset_index(drop=True)
    end_point = len(finaldf)
    x = end_point - forecast_length
    finaldf_train = finaldf.loc[:x - 1, :]
    finaldf_train_x = finaldf_train.loc[:, finaldf_train.columns != target]
    finaldf_train_y = finaldf_train[target]

    fit = model.fit(finaldf_train_x, finaldf_train_y)
    yhat = []
    end_point = len(finaldf)
    df3_end = len(df3)
    for i in range(forecast_length, 0, -1):
        y = end_point - i
        inputfile = finaldf.loc[y:end_point, :]
        inputfile_x = inputfile.loc[:, inputfile.columns != target]
        pred_set = inputfile_x.head(1)
        pred = fit.predict(pred_set)
        df3.at[df3.index[df3_end - i], 'Close'] = pred[0]
        finaldf = df1.drop('Date', axis=1)
        finaldf = finaldf.reset_index(drop=True)
        yhat.append(pred)
    yhat = np.array(yhat)
    return yhat

In [17]:
def predictions(df_coin, forecast_lenght = 5, train_lenght = 100,target = 'Close'):
    """ df_coin must be with date in index,
        forecast_lenght is the amount of days that we will predict
        model is the model predefined to use to get our predictions
        train_length is the amount of days that we will use to train the model
        target is what we are predicting
        This will return a graphic that will contain the data from the train set and our predictions
    """
    df_coin = df_coin.tail(train_lenght)
    df_coin.reset_index(inplace=True)
    df_coin['Date'] = pd.to_datetime(df_coin['Date'], format='%Y-%m-%d')

    df_test = df_coin.copy()
    df_validation = df_test.tail(forecast_lenght)
    df_test.drop(df_test.tail(forecast_lenght).index, inplace = True)

    #Models
    model_rf = RandomForestRegressor(random_state=10)
    model_gb = GradientBoostingRegressor(random_state = 10)
    model_xgb = XGBRegressor(random_state = 10)
    #Metrics
    forecast_test_rf = forecasting(model_rf,df_test,forecast_lenght,target)
    forecast_test_gb = forecasting(model_gb,df_test,forecast_lenght,target)
    forecast_test_xgb = forecasting(model_xgb,df_test,forecast_lenght,target)

    forecast_test = (forecast_test_rf+forecast_test_gb+forecast_test_xgb)/3

    value_mae = mean_absolute_error(df_validation[target], forecast_test)
    value_mse = mean_squared_error(df_validation[target], forecast_test)
    value_r2 = r2_score(df_validation[target], forecast_test)
    value_rmse = math.sqrt(value_mse)

    fig2 = go.Figure()
    fig2.add_trace(go.Scatter(x=df_coin['Date'], y=df_coin[target], 
                    name='Actual Values', mode='lines',line=dict(color='black')))
   
    #Predictions
    forecast_rf = forecasting(model_rf,df_coin,forecast_lenght,target)
    forecast_gb = forecasting(model_gb,df_coin,forecast_lenght,target)
    forecast_xgb = forecasting(model_xgb,df_coin,forecast_lenght,target)
    #ensamble
    forecast = (forecast_rf+forecast_gb+forecast_xgb)/3
    #df that will contain the predictions
    df_pred = pd.DataFrame(columns=['Date',target])
    #Adding the predictions to our dataset
    for day, x in enumerate(forecast):
        new_row={'Date':df_coin['Date'].max() + timedelta(days=1+day),
     target:x[0]}
        df_pred = df_pred.append(new_row, ignore_index=True)

    df_pred['Date'] = pd.to_datetime(df_pred['Date'], format='%Y-%m-%d')

    fig2.add_trace(go.Scatter(x=df_pred['Date'], y=df_pred[target], name='Predictions', mode='lines',line=dict(color='red')))
    fig2.update_layout(dict(updatemenus=[
                        dict(
                        type = "buttons",
                        direction = "left",
                        buttons=list([
                                dict(
                                args=["visible", "legendonly"],
                                label="Deselect All",
                                method="restyle"
                                ),
                                dict(
                                args=["visible", True],
                                label="Select All",
                                method="restyle"
                                )
                        ]),
                        pad={"r": 10, "t": 10},
                        showactive=False,
                        x=1,
                        xanchor="right",
                        y=1.1,
                        yanchor="top"
                        ),
                ]
        ))
    fig2.show()
    return(value_mse, value_rmse, value_mae)

In [18]:
import dash
from dash import dcc, dash_table
from dash import html
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import plotly.express as px
import dash_bootstrap_components as dbc
import yfinance as yf
from datetime import datetime, timedelta
import talib as tb

df_coin = yf.download('SOL-USD',
                      end=datetime.today() - timedelta(days=1), 
                      progress=False,
    )
def df_converter(df): 
    df_sp500 = yf.download('^GSPC', 
                      start='2017-11-09', 
                      end='2022-05-08', 
                      progress=False)
    df_dollar = yf.download('DX=F', 
                      start='2017-11-09', 
                      end='2022-05-08', 
                      progress=False)
    # clearing dollar and sp500 df
    df_dollar.drop(['Open', 'High', 'Low', 'Adj Close', 'Volume'], axis=1, inplace=True)
    df_dollar.rename(columns={"Close": "dollar_close"}, inplace=True)
    df_sp500.drop(['Open', 'High', 'Low', 'Adj Close', 'Volume'], axis=1, inplace=True)
    df_sp500.rename(columns={"Close": "sp500_close"}, inplace=True)
    # clearing general df
    #df_eth.drop('Unnamed: 0', axis=1, inplace=True)
    #df.drop('adj_close', axis=1, inplace=True)
    df.index = df.index.astype('datetime64[ns]')
    # MA df
    df_ma = df['Close'].to_frame()
    df_ma['SMA30'] = df_ma['Close'].rolling(15).mean()
    df_ma['CMA30'] = df_ma['Close'].expanding().mean()
    df_ma['EMA30'] = tb.EMA(df_ma['Close'], timeperiod=15)
    df_ma.dropna(inplace=True)
    # Stoch df
    slowk, slowd = tb.STOCH(df["High"], df["Low"], df["Close"], fastk_period=5, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)
    df_stoch = pd.DataFrame(index=df.index,
                                data={"slowk": slowk,
                                    "slowd": slowd})
    df_stoch.dropna(inplace=True)
    # for later use in the concat
    stoch_c = ['slowk', 'slowd']
    # MACD df 
    macd, macdsignal, macdhist = tb.MACD(df.Close, fastperiod=12, slowperiod=26, signalperiod=9)
    df_macd = pd.DataFrame(index=df.index,
                            data={"macd": macd,
                                  "macdsignal": macdsignal,
                                  "macdhist": macdhist})
    df_macd.dropna(inplace=True)
    # for later use in the concat
    macd_c = ['macd', 'macdsignal', 'macdhist']
    # bb df
    upper, middle, lower = tb.BBANDS(df["Close"], timeperiod=15)
    df_bands = pd.DataFrame(index=df.index,
                                data={"bb_low": lower,
                                    "bb_ma": middle,
                                    "bb_high": upper})
    df_bands.dropna(inplace=True)
    # for later use in the concat
    bands_c = ['bb_low', 'bb_ma', 'bb_high']
    # rsi df
    rsi = tb.RSI(df['Close'], timeperiod=15)
    df_rsi = pd.DataFrame(index=df.index,
                            data={"close": df['Close'],
                                  "rsi": rsi})

    df_rsi.dropna(inplace=True)
    #stdev df
    stdev = tb.STDDEV(df['Close'], timeperiod=15, nbdev=1)
    df_stdev = pd.DataFrame(index=df.index,
                            data={"close": df['Close'],
                                  "stdev": stdev})
    df_stdev.dropna(inplace=True)
    # adx df
    adx = tb.ADX(df['High'], df['Low'], df['Close'], timeperiod=15)
    df_adx = pd.DataFrame(index=df.index,
                                data={"close": df['Close'],
                                    "adx": adx})

    df_adx.dropna(inplace=True)

    # concat 
    result =pd.concat([df, df_ma[['SMA30','CMA30','EMA30']], df_adx['adx'], df_bands[bands_c], df_macd[macd_c], df_rsi['rsi'], df_stdev['stdev'], df_stoch[stoch_c], df_dollar['dollar_close'], df_sp500['sp500_close']], axis=1)
    result.fillna(method='ffill', inplace=True)
    result.dropna(inplace=True)

    return result 

df_coin = df_converter(df_coin)
df_coin

,Open,High,Low,Close,Adj Close,Volume,SMA30,CMA30,EMA30,adx,...,bb_high,macd,macdsignal,macdhist,rsi,stdev,slowk,slowd,dollar_close,sp500_close
Date,,,,,,,,,,,,,,,,,,,,,
2020-05-13,0.538372,0.564874,0.518038,0.550532,0.550532,2.326446e+06,0.630456,0.658397,0.603585,21.590463,...,0.773171,-0.041228,-0.028529,-0.012699,36.035358,0.071357,23.592362,18.768069,100.291000,2820.000000
2020-05-14,0.550532,0.580946,0.550110,0.569118,0.569118,2.047897e+06,0.618675,0.655847,0.599277,22.187288,...,0.750072,-0.039807,-0.030785,-0.009022,38.130968,0.065699,40.929769,26.836762,100.507004,2852.500000
2020-05-15,0.569193,0.574620,0.547093,0.559752,0.559752,1.850070e+06,0.609922,0.653177,0.594336,22.784645,...,0.738327,-0.038987,-0.032425,-0.006561,37.468194,0.064203,56.470298,40.330810,100.433998,2863.699951
2020-05-16,0.558981,0.606534,0.554361,0.599120,0.599120,2.300409e+06,0.601277,0.651716,0.594934,22.556816,...,0.712865,-0.034759,-0.032892,-0.001867,42.007706,0.055794,74.794339,57.398135,100.433998,2863.699951
2020-05-17,0.595228,0.640196,0.589175,0.612989,0.612989,2.936428e+06,0.594418,0.650697,0.597191,21.610798,...,0.688217,-0.029945,-0.032303,0.002358,43.554393,0.046899,80.809923,70.691520,100.433998,2863.699951
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-05-22,50.320660,53.021660,49.862785,52.755180,52.755180,1.105761e+09,55.021761,56.982347,58.130016,44.655632,...,70.562499,-10.944958,-11.448725,0.503767,37.165800,7.770369,28.435353,25.943267,103.694000,4123.339844
2022-05-23,52.736088,54.474648,48.927246,49.056850,49.056850,1.661669e+09,53.277539,56.972107,56.995870,44.561843,...,64.681031,-10.665722,-11.292124,0.626402,35.070199,5.701746,29.044400,27.101726,103.694000,4123.339844
2022-05-24,49.064232,50.577522,47.500824,49.653336,49.653336,1.508167e+09,52.369784,56.962664,56.078053,44.610040,...,62.549252,-10.277818,-11.089263,0.811445,35.696748,5.089734,31.765465,29.748406,103.694000,4123.339844


In [20]:
predictions(df_coin=df_coin, forecast_lenght = 15, train_lenght = 500,target = 'Close')

(1197.5982591919628, 34.60633264580289, 33.30507032342677)